In [ ]:
import os, re, pandas as pd, numpy as np, gensim, matplotlib.pyplot as plt, warnings, spacy
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

warnings.filterwarnings("ignore")

In [ ]:
def read_data():
    """Permite leer la información del WS, ajustar encoding convenientemente. 
       # TODO Encoding de raw data
    Returns:
        df (DataFrame()): Genera un solo DF con el compilado de todas las fuentes ".csv"
    """
    fuentes = list(pd.Series(os.listdir('../data/raw/'))[pd.Series(os.listdir('../data/raw/')).str.contains('.csv')])
    df = pd.DataFrame(columns=['Fecha Extraccion', 'Titulo', 'Fecha Publicacion',
                               'Resumen', 'URL','Imagen', 'Empresa', 'Contenido',
                               'Tags', 'Tema', 'Autor','RelNewsUrls', 'Fuente'])
    for fuente in fuentes:
        if (fuente == "larepublica.csv"):
            df_ = pd.read_csv("../data/raw/"+fuente, sep = ',', encoding = 'latin_1')
            df_['Fuente'] = fuente
        else:
            df_ = pd.read_csv("../data/raw/"+fuente, sep = ',', encoding = None)
            df_['Fuente'] = fuente
        df = pd.concat([df, df_], axis=0)
    # df = df[df['Fuente'] == 'semana.csv'] # TODO quitar linea al arreglar encoding de la republica
    df = df.drop(df[df['Contenido'] == "SIN PARRAFOS"].index).reset_index(drop=True)
    return df

In [ ]:
es = spacy.load('es_core_news_lg')
stop_words = es.Defaults.stop_words
# stop_words.extend([]) # Agregar palabras de ser necesario
a,b = 'áéíóúü','aeiouu'
trans = str.maketrans(a,b)

In [ ]:
def pre_proc_text(df_, tipo, cliente_):
    '''
    Estandariza y elimina bugs de cada texto
    Args:
        df_ (Object): DataFrame resultado del WS
        tipoc (Str): Tipo de segmento de informacion a analizar ("Titulo", "Resumen", "Contenido")
        cliente_ (Str): Empresa clientes foco para analisis
    output:
        lemma_words (List): Lista con palabras lematizadas
        lda_words: 
    '''
    # Filtrado de informacion para analisis
    texto = df_[
        (df_.Empresa==cliente_)
    ][tipo]
    
    texts = texto.str.lower()
    texts = texts.str.replace(cliente_.lower(), '', regex=True)
    texts = texts.str.replace('[0-9]', '', regex=True)
    texts = texts.str.replace('\W', ' ', regex=True).str.strip()
    texts = texts.str.replace(' {2,}', ' ', regex=True)
    texts = texts.apply(lambda x: x.translate(trans))
    texts = texts.apply(
        lambda x: ' '.join(pd.Series(re.split(' ', x))[~pd.Series(re.split(' ', x)).isin(stop_words)].tolist())
    )
    lda_words = texts.str.split('\W')
    return lda_words

In [ ]:
def useLDA(texts, n):
    """ Ejecuta LDA por texto analizado
    Args:
        texts (Object): Texto luego de pre-procesamiento
        n (int): Numero de topics a encontrar
    """
    dictionary = Dictionary(texts)
    bow_corpus = [dictionary.doc2bow(doc) for doc in texts]
    lda_model = LdaMulticore(bow_corpus, num_topics=n, id2word=dictionary, passes=2, workers=2)
    string = ''
    return lda_model.show_topics(num_topics=1, num_words=5, log=False, formatted=False)  # show_topics()[0][1]
    # for idx, topic in lda_model.print_topics(-1):
    #     string += '*'*50+'\n'
    #     string += f'Topic: {idx} \nWords: {topic}'+'\n'
    # print(string)

In [ ]:
data = read_data()
data_aux = []
for i in range(data.shape[0]):
    aux = []
    for item in list(useLDA(texts = pre_proc_text(df_ = data, tipo = 'Contenido', cliente_ = "Ecopetrol").loc[[i,]], n = 3)[0][1]):
        aux.append(item[0])
    data_aux.append(aux)
data['Topics'] = data_aux

In [ ]:
data.to_csv("../data/curated/curated_data.csv",index=False)